In [15]:
import pandas as pd
data = pd.read_csv(r"C:\Users\Administrator\Desktop\机器学习\幸存者.csv"
                   ,index_col=0
                   ,encoding='gb18030')
 
data.head()

,Age,Sex,Sur
ID,,,
0,3.0,M,Y
1,21.0,F,N
2,43.0,F,N
3,32.0,M,N
4,12.0,NaN,N


In [16]:
data.info()#查看数据的基本情况本来有10个，可是年龄和性别均有一个缺失值。

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     9 non-null      float64
 1   Sex     9 non-null      object 
 2   Sur     10 non-null     object 
dtypes: float64(1), object(2)
memory usage: 320.0+ bytes


In [17]:
#填补年龄
Age = data.loc[:,"Age"].values.reshape(-1,1)            #sklearn当中特征矩阵必须是二维
'''reshape(-1,1):提取出来的年龄是一维数组，但是必须转变为2D.loc[:,"年龄"]：提取所有行的年龄这一列。'''

Age[:20]#显示前20行！
 

array([[ 3.],
       [21.],
       [43.],
       [32.],
       [12.],
       [35.],
       [nan],
       [45.],
       [22.],
       [64.]])

In [18]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer()                              #实例化，默认均值填补
imp_median = SimpleImputer(strategy="median")           #用中位数填补
imp_0 = SimpleImputer(strategy="constant",fill_value=0) #用0填补
 

In [19]:
imp_mean = imp_mean.fit_transform(Age)                  #fit_transform一步完成调取结果
imp_median = imp_median.fit_transform(Age)
imp_0 = imp_0.fit_transform(Age)
 

In [20]:
imp_mean[:20]

array([[ 3.        ],
       [21.        ],
       [43.        ],
       [32.        ],
       [12.        ],
       [35.        ],
       [30.77777778],
       [45.        ],
       [22.        ],
       [64.        ]])

In [21]:
imp_median[:20]

array([[ 3.],
       [21.],
       [43.],
       [32.],
       [12.],
       [35.],
       [32.],
       [45.],
       [22.],
       [64.]])

In [22]:
imp_0[:20]

array([[ 3.],
       [21.],
       [43.],
       [32.],
       [12.],
       [35.],
       [ 0.],
       [45.],
       [22.],
       [64.]])

In [23]:
#在这里我们使用中位数填补Age
data.loc[:,"Age"] = imp_median
 
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     10 non-null     float64
 1   Sex     9 non-null      object 
 2   Sur     10 non-null     object 
dtypes: float64(1), object(2)
memory usage: 320.0+ bytes


In [24]:
#使用众数填补Sex，因为这一属性非数字是文字！
Sex = data.loc[:,"Sex"].values.reshape(-1,1)
imp_mode = SimpleImputer(strategy = "most_frequent")
data.loc[:,"Sex"] = imp_mode.fit_transform(Sex)
 
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     10 non-null     float64
 1   Sex     10 non-null     object 
 2   Sur     10 non-null     object 
dtypes: float64(1), object(2)
memory usage: 320.0+ bytes


### 处理分类型特征：编码与哑变量

### 1.preprocessing.LabelEncoder：标签专用，能够将分类转换为分类数值

In [25]:
from sklearn.preprocessing import LabelEncoder
 
y = data.iloc[:,-1]                         #要输入的是标签，不是特征矩阵，所以允许一维

In [26]:
 
le = LabelEncoder()                         #实例化
le = le.fit(y)                              #导入数据
label = le.transform(y)                     #transform接口调取结果

label

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1])

In [27]:
le.classes_                                 #属性.classes_查看标签中究竟有多少类别

array(['N', 'Y'], dtype=object)

In [28]:
le.fit_transform(y)   #注意一步到位的方法之后就看不了le.classes_。 #也可以直接fit_transform一步到位

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1])

In [30]:
le.inverse_transform(label)                 #使用inverse_transform可以逆转

array(['Y', 'N', 'N', 'N', 'N', 'N', 'N', 'Y', 'N', 'Y'], dtype=object)

In [31]:
data.iloc[:,-1] = label  #让文字标签等于我们运行出来的结果数字。
 
data.head()

,Age,Sex,Sur
ID,,,
0,3.0,M,1
1,21.0,F,0
2,43.0,F,0
3,32.0,M,0
4,12.0,M,0


In [34]:
 
#如果不需要教学展示的话我会这么写：
from sklearn.preprocessing import LabelEncoder
#令最后一行文字转换为数字。
data.iloc[:,-1] = LabelEncoder().fit_transform(data.iloc[:,-1])
data

,Age,Sex,Sur
ID,,,
0,3.0,M,1
1,21.0,F,0
2,43.0,F,0
3,32.0,M,0
4,12.0,M,0
5,35.0,M,0
6,32.0,M,0
7,45.0,F,1
8,22.0,F,0


### 2. preprocessing.OrdinalEncoder：特征专用，能够将分类特征转换为分类数值


In [35]:
from sklearn.preprocessing import OrdinalEncoder
 
#接口categories_对应LabelEncoder的接口classes_，一模一样的功能
data_ = data.copy()
data_


,Age,Sex,Sur
ID,,,
0,3.0,M,1
1,21.0,F,0
2,43.0,F,0
3,32.0,M,0
4,12.0,M,0
5,35.0,M,0
6,32.0,M,0
7,45.0,F,1
8,22.0,F,0


In [36]:
#data_.head()
OrdinalEncoder().fit(data_.iloc[:,1:-1]).categories_#查看每个文字属性下的类

[array(['F', 'M'], dtype=object)]

In [37]:
data_.iloc[:,1:-1] = OrdinalEncoder().fit_transform(data_.iloc[:,1:-1])
#令第一列到最后一列中文字属性转换为数字。
 
data_.head()

,Age,Sex,Sur
ID,,,
0,3.0,1.0,1
1,21.0,0.0,0
2,43.0,0.0,0
3,32.0,1.0,0
4,12.0,1.0,0
